In [1]:
from dotenv import load_dotenv
load_dotenv('.env')

True

In [2]:

import os

from utils.fundamentals import get_fundamentals
from utils.industry import get_industry_classification
from utils.pandas_utils import merge_df_safely
from utils.rating import get_ratings
from utils.technical import get_technicals
from utils.tradingview import TradingView

OCI FS Configured


In [3]:
df = TradingView.get_base_symbols()
print("Base Symbols loaded")

Base Symbols loaded


In [4]:
df = TradingView.get_base_symbols()
print("Base Symbols loaded")


Base Symbols loaded


In [5]:
index_df = await  TradingView.get_index(df.columns)
print("Index loaded")

Started: 1/1
Completed: 1/1
Index loaded


In [6]:
df = merge_df_safely(df, get_industry_classification())
print("Industry classification updated")

Industry classification updated


In [7]:
df = merge_df_safely(df, get_fundamentals())
print("Fundamentals updated")


Fundamentals updated


In [8]:
df = merge_df_safely(df, await get_technicals(df, df.index.to_list()))
print("Technicals updated")


Started: 1/1
Completed: 1/1
Started: 1/5
Completed: 1/5
Started: 2/5
Completed: 2/5
Started: 3/5
Completed: 3/5
Started: 4/5
Completed: 4/5
Started: 5/5
Completed: 5/5
Technicals updated


In [49]:
import pandas as pd
def sector_industry_strength_rating2(df: pd.DataFrame) -> pd.DataFrame:
    """
    Computes group strength rankings based on median returns for valid symbols.

    - For each timeframe (e.g. 3M), filters symbols with price_volume over 1 Cr
    - Excludes type == 'stock'
    - Requires groups with ≥ 2 valid symbols
    - Adds ranking and return columns for each group (sector, industry, etc.)

    Returns:
        pd.DataFrame with *_ranking_<timeframe>, *_return_<timeframe> columns.
    """
    df = df.copy()
    min_liquidity = 1 * 10 ** 7  # 1 Cr

    perf_cols = [
        "price_perf_1D", "price_perf_1W", "price_perf_1M",
        "price_perf_3M", "price_perf_6M", "price_perf_9M", "price_perf_12M"
    ]

    volume_map = {
        "1D": "price_volume",
        "1W": "price_volume_sma_5D",
        "1M": "price_volume_sma_21D",
        "3M": "price_volume_sma_63D",
        "6M": "price_volume_sma_126D",
        "9M": "price_volume_sma_189D",
        "12M": "price_volume_sma_252D"
    }

    def compute_group_rankings(df_all, group_col):
        result_dict = {}

        for perf_col in perf_cols:
            timeframe = perf_col.split('_')[-1]
            volume_col = volume_map.get(timeframe)
            if volume_col not in df_all.columns:
                continue

            valid_df = df_all[
                (df_all["type"].str.lower() == "stock") &
                (df_all[volume_col] > min_liquidity)
            ].copy()

            group_counts = valid_df[group_col].value_counts()
            valid_groups = group_counts[group_counts >= 2].index
            valid_df = valid_df[valid_df[group_col].isin(valid_groups)]

            return_col = f"{group_col}_return_{timeframe}"
            rank_col = f"{group_col}_ranking_{timeframe}"

            group_median = valid_df.groupby(group_col)[perf_col].median().rename(return_col)
            group_rank = group_median.rank(method='min', ascending=False).astype(pd.Int64Dtype()).rename(rank_col)

            result_dict[return_col] = group_median
            result_dict[rank_col] = group_rank

        return pd.concat(result_dict.values(), axis=1)

    def merge_group_info(df_base, group_col):
        group_metrics = compute_group_rankings(df_base, group_col)
        df_merged = df_base.merge(group_metrics, how='left', left_on=group_col, right_index=True)
        return df_merged

    # Apply to each group level
    df_result = df.copy()
    for group_col in ['sector', 'industry', 'sub_industry', 'industry_2']:
        df_result = merge_group_info(df_result, group_col)

    # Fill missing values
    for group_col in ['sector', 'industry', 'sub_industry', 'industry_2']:
        for col in df_result.columns:
            if col.startswith(f"{group_col}_return_"):
                df_result[col] = df_result[col].fillna(-9999)
            elif col.startswith(f"{group_col}_ranking_"):
                df_result[col] = df_result[col].fillna(9999).astype(pd.Int64Dtype())

    return df_result

In [51]:
sector_industry_strength_rating2(df)[['industry','industry_ranking_1M']]

/var/folders/6j/dvdvpwvj2gq3fdjs2dw420c40000gn/T/ipykernel_23920/3993359122.py:53: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_median = valid_df.groupby(group_col)[perf_col].median().rename(return_col)
/var/folders/6j/dvdvpwvj2gq3fdjs2dw420c40000gn/T/ipykernel_23920/3993359122.py:53: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_median = valid_df.groupby(group_col)[perf_col].median().rename(return_col)
/var/folders/6j/dvdvpwvj2gq3fdjs2dw420c40000gn/T/ipykernel_23920/3993359122.py:53: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pan

,industry,industry_ranking_1M
ticker,,
NSE:RELIANCE,Petroleum Products,25
NSE:HDFCBANK,Banks,51
NSE:TCS,IT - Software,10
NSE:AIRTELPP.E1,Wireless telecommunications,9999
NSE:BHARTIARTL,Telecom - Services,43
...,...,...
NSE:AJOONI,Food Products,54
NSE:MYMUDRA,Finance/Rental/Leasing,8
NSE:BRACEPORT,Air freight/Couriers,9999


In [ ]:
df = merge_df_safely(df, get_ratings(df))
print("Rating updated")